# Camera Calibration

In this notebook, I aim to get familiar with camera Calibration and how to use it to undistort images.

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import glob

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# (points_per_row,points_per_colum)
points_per_row = 6
points_per_colum = 9
pattern_size = (points_per_row, points_per_colum)

In [2]:
# prepare object points
objp = np.zeros((points_per_row * points_per_colum, 3), np.float32)
objp[:, :2] = np.mgrid[0:points_per_row, 0:points_per_colum].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images
objpoints = []  # 3D points in real-world space
imgpoints = []  # 2D points in image plane

# Get all JPEG files in the current directory
image_files = glob.glob('*.jpeg')

for image_file in image_files:
    # Read the image
    img = cv.imread(image_file)

    # Convert img to grayscale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv.findChessboardCorners(gray, pattern_size, None)

    if ret:
        # If corners are found, add object points and image points
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, pattern_size, corners2, ret)

        # wait a bit
        cv.waitKey(1000)

    # Display the img
    cv.imshow('Image', img)

    # Wait for key press
    if cv.waitKey(5) == ord('q'):
        break

# Release the capture and destroy windows
cv.destroyAllWindows()

In [6]:
# Calibrate camera
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# undistort image
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
cv.imwrite('calibresult.png', dst)

True

In [7]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.09902093682900776


# Google media pip research

In [28]:
import time
import cv2
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
PoseLandmarkerResult = mp.tasks.vision.PoseLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

aa_output_image = None

def print_result(result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    # global aa_output_image
    # print('pose landmarker result: {}'.format(result))
    # aa_output_image = output_image
    cv2.imshow('frame', output_image.numpy_view())

model_path = "/Users/Ethan/Developer/Projects/College/大四下/人工智慧/final/lotpose_bd/lotpose/checkpoints/pose_landmarker_lite.task"
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result
)

cap = cv2.VideoCapture(2)

landmarker = PoseLandmarker.create_from_options(options)
while True:
    ret, frame = cap.read()
    cv2.imshow('frame', frame)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    landmarker.detect_async(mp_image, int(time.time() * 1000))
    if cv2.waitKey(5) == ord('q'):
        landmarker.close()
        break

KeyboardInterrupt: 